# ⚙️ Configuration

In [ ]:
# pip install --upgrade  domolibrary

## ⚙️ Auth Object
Configure a `DomoAuth` object that will be used to interact with `DomoDatasets`

For this project we must use `DomoFullAuth` because the bootstrap API will not accept token auth

In [ ]:
import os

DOMO_INSTANCE = "domo-community"
DOMO_USERNAME = os.environ["DOMO_USERNAME"]
DOMO_PASSWORD = os.environ["DOJO_PASSWORD"]
FEATURES_DATASET_ID = "44c5af30-ea04-49e4-9d7a-529afd223590"

In [ ]:
import domolibrary.client.DomoAuth as dmda


async def generate_auth(domo_instance, domo_username, domo_password) -> dmda.DomoAuth:
    """receive auth credentials, generate and test an auth object"""

    print(f"⚙️ generate_auth:  retrieving config data from {domo_instance} ⚙️")

    auth = dmda.DomoFullAuth(
        domo_username=domo_username,
        domo_password=domo_password,
        domo_instance=domo_instance,
    )

    # confirm we can get a valid auth token
    await auth.get_auth_token()

    if not auth.token:
        raise Exception("💀 generate_auth: Error retrieving Auth Token")

    if auth.token:
        print("🌼 generate_auth:  success retrieving Auth Token")

    return auth


# test
test_auth = await generate_auth(
    domo_instance=DOMO_INSTANCE,
    domo_password=DOMO_PASSWORD,
    domo_username=DOMO_USERNAME,
)

## ⚙️ Dataset Object
Configure a `DomoDataset` that will receive the Bootstrap dataframe from `DomoBootstrap`

Note, the schema of the dataset must match the schema of the dataframe being uploaded or the dataset won't index.
You can alter schema using the "Schema Management" tool from the Domo Governance Toolkit or java cli.

In [ ]:
import domolibrary.classes.DomoDataset as dmds


async def generate_dataset(dataset_id: str, auth: dmda.DomoAuth):
    """generates a DomoDataset class object from a dataset_id"""

    return await dmds.DomoDataset.get_from_id(dataset_id=dataset_id, auth=auth)


# test
await generate_dataset(dataset_id=FEATURES_DATASET_ID, auth=test_auth)

## 💾 Bootstrap Data
Retrieve your feature list using `DomoBootstrap.get_features()` and format it as a dataframe

In [ ]:
import pandas as pd
import domolibrary.classes.DomoBootstrap as dmbsr


async def get_bootstrap(auth, debug_api: bool = False) -> pd.DataFrame:
    """generates a dataframe of bootstrap_features"""

    domo_bsr = dmbsr.DomoBootstrap(auth=auth)

    bsr_features = await domo_bsr.get_features(debug_api=debug_api)

    bsr_features_df = pd.DataFrame(bsr_features)
    bsr_features_df["instance"] = auth.domo_instance

    return bsr_features_df


# test
upload_df = await get_bootstrap(test_auth)
upload_df[0:5]

# 🚀 Execute Main 🚀

In [ ]:
import asyncio
import pandas as pd
import time

import domolibrary.utils.chunk_execution as ce
import domolibrary.utils.upload_data as upd


async def main(debug_api: bool = False):
    script_start = time.time()

    config_auth = await generate_auth(
        domo_instance=DOMO_INSTANCE,
        domo_password=DOMO_PASSWORD,
        domo_username=DOMO_USERNAME,
    )

    upload_df = await get_bootstrap(auth=config_auth, debug_api=debug_api)

    feature_ds = await generate_dataset(
        dataset_id=FEATURES_DATASET_ID, auth=config_auth
    )

    print(
        f"🥫 {len(upload_df)} rows to upload to {feature_ds.name} - {feature_ds.display_url()}"
    )

    await feature_ds.upload_data(
        upload_df=upload_df,
        partition_key=config_auth.domo_instance,
        is_index=True,
        dataset_id=FEATURES_DATASET_ID,
        auth=config_auth,
    )

    script_end = time.time()
    print(f"🥫 done!  Took {script_end - script_start} seconds")


await main(debug_api=False)

🥫 done!  Took 11.374664545059204 seconds


# Results #

In [ ]:
pd.set_option("display.max_rows", None)
upload_df[0:10]

,id,name,label,type,purchased,enabled,instance
0,4,search,,STANDARD,False,True,domo-community
1,5,launcher,,PREMIUM,True,True,domo-community
2,9,profile-reminder,,STANDARD,False,True,domo-community
3,17,enableSwapDatasource,,STANDARD,False,True,domo-community
4,19,up,,STANDARD,True,True,domo-community
5,25,account-management,,STANDARD,False,True,domo-community
6,66,advanced-scheduling,,STANDARD,False,True,domo-community
7,67,2factor,,STANDARD,False,True,domo-community
8,69,saasaas,,STANDARD,False,True,domo-community
9,1466,access-token,,STANDARD,False,True,domo-community
